Going through experimental data collected in lab using different material substrates and coatings 
 used for soft neural interface device
with Ben Rees

--------------------

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1. Test bulk substrates (PI, silicone, kirigami PI). Plot engineering stress–strain curves to failure at a
fixed displacement rate and compare mechanical properties. Include elongation limit, elastic modulus,
tensile strength, and toughness.

**sample info**
|           Sample Name          | Sample Width (mm) | Sample Thickness (µm) | Original Length (mm) | Original Extension (µm) |
|:------------------------------:|:-----------------:|:---------------------:|:--------------------:|:-----------------------:|
|            Polyimide           |       5.175       |           15          |          19          |           53.8          |
|           PI Kirigami          |                   |          25.4         |         23.79        |           4838          |
|              PDMS              |        5.6        |         1024.5        |         22.38        |          3588.7         |
| SiC 2 µm on 15 µm PI (N0018-2) |      4.96625      |           17          |        21.195        |           46.1          |
|  SiC 1 µm on 15 µm PI (NM5-1)  |       4.8575      |           16          |        21.395        |           31.7          |

2. Test bilayer films (SiC on PI). Compare the stress–strain behavior to that of the bulk PI substrate. Estimate the coating (SiC) modulus and compare it with literature values for the encapsulation material. If errors occur in extracting the coating modulus, briefly explain likely causes.

3. Cracking analysis from recordings (SiC on PI): Include image arrays from the recording files at 0% strain and Stages I, II, III. Calculate and report crack-onset strain, εc, critical energy release rate, Gc, crack density at saturation CDsat, critical stress-transfer length lc, and interfacial shear strength (IFSS). Compare the adhesion level with other film-substrate interface materials.

4. Nanocomposite comparison [no test; please use files provided]: Review the recordings for MMT nanoclay-PVA nanocomposites on PET. Describe how their crack evolution differs from SiC on PI. Show Stages I–III. If the stages are not clearly distinguishable, discuss possible reasons.